In [1]:
from sklearn import datasets, metrics, cross_validation
import tensorflow as tf
from tensorflow.contrib import layers
from tensorflow.contrib import learn

def my_model(features, target):
    target_classes = 4
    #target = tf.one_hot(target, target_classes, 1., 0.)
    #features = tf.one_hot(features, feature_classes, 1, 0)
    features = layers.stack(features, layers.fully_connected, [10, 20, 10])
    
    #Use `tf.contrib.losses.softmax_cross_entropy` and explicit logits computation.
    # Compute logits (1 per class) and compute loss.
    logits = layers.fully_connected(features, target_classes)
    #loss = tf.contrib.losses.softmax_cross_entropy(logits, target)
    
    #loss = tf.nn.softmax_cross_entropy_with_logits(logits, target)
    print('logits/labels:', logits.get_shape(), target.get_shape())
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits, target))
    
    #prediction, loss = (
    #    tf.contrib.learn.models.logistic_regression_zero_init(features, target)
    #)
    train_op = tf.contrib.layers.optimize_loss(
        loss, tf.contrib.framework.get_global_step(), optimizer='Adagrad',
        learning_rate=0.1)
    #return {'class': tf.argmax(prediction, 1), 'prob': prediction}
    return tf.argmax(logits, 1), loss, train_op

iris = datasets.load_iris()
x_train, x_test, y_train, y_test = cross_validation.train_test_split(
  iris.data, iris.target, test_size=0.2, random_state=35)

classifier = learn.SKCompat(learn.Estimator(model_fn=my_model))
classifier.fit(x_train, y_train, steps=700)

predictions = list(classifier.predict(x_test)) #TODO undo onhot?
print("Accuracy:", metrics.accuracy_score(y_test, predictions))

/home/jason/.local/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fdc242a1e48>, 'save_checkpoints_steps': None, '_environment': 'local', 'keep_checkpoint_max': 5, 'save_summary_steps': 100, 'keep_checkpoint_every_n_hours': 10000, '_is_chief': True, '_task_id': 0, '_num_ps_replicas': 0, '_evaluation_master': '', '_task_type': None, 'save_checkpoints_secs': 600, '_master': '', 'tf_random_seed': None, 'tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
}
logits/labels: (?, 4) (?,)
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:loss = 1.35926142788, step = 1
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmp7ad7x0q2/model.ckpt.
INFO:tensorflow:loss = 0.170548893944, step = 101
INFO:tensorflow:global_step/sec: 282.408
INFO:tensorflow:loss = 0.0828249298931, step = 201
INFO:tensorflow:global_step/sec: 852.568
INFO:tensorflow:loss = 0.0691876533619, step = 301
INFO:tensorflow:glo